## Reconstruction algorithm

[1] M. Humbert, F. Wagner, H. Moustahfid, and C. Esling, “Determination of the Orientation of a Parent β Grain from the Orientations of the Inherited α Plates in the Phase Transformation from Body-Centred Cubic to Hexagonal Close Packed,” J. Appl. Crystallogr., vol. 28, pp. 571–576, 1995.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from defdap.quat import Quat
import defdap.ebsd as ebsd

from defdap.plotting import PolePlot, MapPlot

%matplotlib qt

### Load in EBSD file, find grains and calculate grain average orientaions

In [4]:
dropboxPath = "/Users/mbcx9ma4/Dropbox (The University of Manchester)/Beta Reconstruction/"
EbsdFilePath = dropboxPath + "data/triple_point/ZrNb_triplepoint_alpha"
# EbsdFilePath = dropboxPath + "data/large/ZrNb_large_alpha"

EbsdMap = ebsd.Map(EbsdFilePath, "hexagonal")
EbsdMap.buildQuatArray()

EbsdMap.findBoundaries(boundDef=8)
EbsdMap.findGrains(minGrainSize=3)

EbsdMap.calcGrainAvOris()

Loaded EBSD data (dimensions: 285 x 276 pixels, step size: 5.0 um)
Done                                               

In [5]:
EbsdMap.locateGrainID(plotGBs=True)

Grain ID: 903
Grain ID: 797
Grain ID: 903
Grain ID: 1350
Grain ID: 903
Grain ID: 1180
Grain ID: 903
Grain ID: 891
Grain ID: 870
Grain ID: 781
Grain ID: 903
Grain ID: 821


In [7]:
EbsdMap.plotGrainMap()

In [8]:
EbsdMap.buildNeighbourNetwork()

In [12]:
EbsdMap.displayNeighbours()

### Construct subset of symmetries and the Burger transformation

In [9]:
hexSymms = Quat.symEqv("hexagonal")
hexFundSymms = [
    hexSymms[0],
    hexSymms[5],
    hexSymms[4],
    hexSymms[2],
    hexSymms[10],
    hexSymms[11]
]

cubicSymms = Quat.symEqv("cubic")
cubicFundSymms = [
    cubicSymms[0],
    cubicSymms[7],
    cubicSymms[9],
    cubicSymms[1],
    cubicSymms[22],
    cubicSymms[16],
    cubicSymms[12],
    cubicSymms[15],
    cubicSymms[4],
    cubicSymms[8],
    cubicSymms[21],
    cubicSymms[20]
]

# HCP -> BCC
eulers = np.array([135., 90., 354.74])*np.pi/180.
burgersQuat = Quat(*eulers)
burgersQuat = burgersQuat.conjugate

### Calculate possible beta orientations for each grain

In [10]:
for grain in EbsdMap:
    grain.betaOris = []

    for symm in hexFundSymms:
        grain.betaOris.append(burgersQuat * symm.conjugate * grain.refOri)


### Calculate possible beta orientations from misorientations between each grain adn its neighbours

In [29]:
import time
start = time.time()


maxDevFromBurgers = 5    # in degrees


cubicSymComps = np.empty((4, len(cubicFundSymms)))
for i, cubicSymm in enumerate(cubicFundSymms):
    cubicSymComps[:, i] = cubicSymm.quatCoef
    

# grainID = 479
# grainID = 457
# grain = EbsdMap[grainID]

numGrains = len(EbsdMap)
for grainID, grain in enumerate(EbsdMap):
# if True:
    if grainID % 10 == 0:
        print("\r Grain number {} of {}".format(grainID+1, numGrains), end="")
    
    grain.possibleBetaOris = []
    grain.betaDeviations = []

    neighbourIDs = list(EbsdMap.neighbourNetwork.neighbors(grainID))
    neighbourGrains = [EbsdMap[i] for i in neighbourIDs]
    
    grainInvOri = grain.refOri.conjugate
    
    for neighbourID, neighbourGrain in zip(neighbourIDs, neighbourGrains):
#         if neighbourID != 479:
#             continue
        
        neighbourGrainOri = neighbourGrain.refOri
        
        a2a1inv = neighbourGrainOri * grainInvOri
        
        mis144 = np.zeros((12, 12))
        RCS144 = np.zeros((12, 12), dtype=int)
        
        # calculate all posible S^B_m (eqn 11. from [1]) from the measured misorientation from 2 neighbour alpha grains
        # for each S^B_m calculate the 'closest' cubic symmetry (from subset) and the deviation from this symmetry
        for i, hexSymm in enumerate(hexSymms):
            dummy = a2a1inv * hexSymm

            for j, hexSymm2 in enumerate(hexSymms):
                Bvariant = burgersQuat * ((hexSymm2.conjugate * dummy) * burgersQuat.conjugate)
                
                misOris = np.einsum("ij,i->j", cubicSymComps, Bvariant.quatCoef)
                
#                 misOris = []
#                 for cubicSymm in cubicFundSymms:                    
#                     misOri = Bvariant.dot(cubicSymm)
#                     misOris.append(misOri)    
#                 misOris = np.abs(np.array(misOris))

                misOris = np.abs(misOris)
                misOris[misOris > 1] = 1.
                misOris = 2 * np.arccos(misOris) * 180 / np.pi
                
                minMisOriIdx = np.argmin(misOris)
                mis144[i, j] = misOris[minMisOriIdx]
                RCS144[i, j] = minMisOriIdx
        
        # find the hex symmetries (i, j) from give the minimum deviation from the burgers relation
        # for the minimum store: the deviation, the hex symmetries (i, j) and the cubic symmetry
        # if the deviation is over a threshold then set cubic symmetry to -1
        minMisOriIdx = np.unravel_index(np.argmin(mis144), mis144.shape)
        devFromBurgers = mis144[minMisOriIdx]
        cubicSymmIndx = RCS144[minMisOriIdx] if devFromBurgers < maxDevFromBurgers else -1
        a1Symm = minMisOriIdx[0]
        a2Symm = minMisOriIdx[1]
#         print(devFromBurgers, cubicSymmIndx)

#         with np.printoptions(precision=2, suppress=True, linewidth=100):
#             print(minMisOriIdx)
#             print(mis144)
#             print(devFromBurgers)
#             print(RCS144)
#             print(cubicSymmIndx)
#             print(" ")

        possibleBetaOris = []
        if cubicSymmIndx > -1 and cubicSymmIndx < 9:
            # one possible beta orientation
            # A:
            possibleBetaOris.append(
                burgersQuat * hexSymms[a1Symm].conjugate * grain.refOri
            )
            
        elif cubicSymmIndx == 9:
            # three possible beta orientation
            # A:
            possibleBetaOris.append(
                burgersQuat * hexSymms[a1Symm].conjugate * grain.refOri
            )
            # B:
            # hexFundSymms[1] is C^+_3z
            possibleBetaOris.append(
                burgersQuat * hexFundSymms[1].conjugate * hexSymms[a1Symm].conjugate * grain.refOri
            )
            # C:
            # hexFundSymms[2] is C^+_6z
            possibleBetaOris.append(
                burgersQuat * hexFundSymms[2].conjugate * hexSymms[a1Symm].conjugate * grain.refOri
            )
            
        elif cubicSymmIndx > 9:
            # two possible beta orientation
            # A:
            possibleBetaOris.append(
                burgersQuat * hexSymms[a1Symm].conjugate * grain.refOri
            )
            # D:
            # hexFundSymms[4] is C'_22
            possibleBetaOris.append(
                burgersQuat * hexFundSymms[4].conjugate * hexSymms[a1Symm].conjugate * grain.refOri
            )
            
        grain.possibleBetaOris.append(possibleBetaOris)
        grain.betaDeviations.append(devFromBurgers)
        
#         print(grainID, neighbourID)
#         print(possibleBetaOris)
#         print(devFromBurgers)
#         break
        
        
    # do all the accounting stuff
    oriTol = 3.  # in degrees
    oriTol = np.cos(oriTol / 2 * np.pi / 180.)  # divide 2 because of 2* in misorientation

    allPossibleBetaOris = [item for sublist in grain.possibleBetaOris for item in sublist]
    uniqueBetaOris = []
    countBetaOris = []
    variantIndexes = []

    for ori in allPossibleBetaOris:
        found = False
        for i, uniqueOri in enumerate(uniqueBetaOris):
            misOri = ori.misOri(uniqueOri, "cubic")
            if misOri > oriTol:
                found = True
                countBetaOris[i] += 1

        if not found:
            uniqueBetaOris.append(ori)
            countBetaOris.append(1)

            for i, betaVariant in enumerate(grain.betaOris):
                misOri = ori.misOri(betaVariant, "cubic")
                if misOri > oriTol:
                    variantIndexes.append(i)
                    break
            else:
                variantIndexes.append(-1)
                print("Couldn't find beta variant. Grain {:}".format(grainID))

    variantCount = [0, 0, 0, 0, 0, 0]
    for i in range(len(variantIndexes)):
        if i > -1:
            variantCount[variantIndexes[i]] = countBetaOris[i]

    grain.variantCount = variantCount

#     print(uniqueBetaOris)
#     print(countBetaOris)
#     print(variantIndexes)
#     print(variantCount)


end = time.time()
print(end - start)

 Grain number 2061 of 2070453.3097519874573


In [30]:
print((end - start) / 60)

7.555162533124288


In [ ]:
7.73 min
7.55 min

### Look at the data stored for each grain

In [34]:
EbsdMap.locateGrainID()

Grain ID: 838


In [31]:
grain = EbsdMap[838]

In [1]:
grain.betaOris

NameError: name 'grain' is not defined

In [33]:
grain.possibleBetaOris

[[[0.4611, -0.8264, 0.3052, 0.1062]],
 [[0.4611, -0.8264, 0.3052, 0.1062]],
 [[0.5921, 0.0287, -0.8046, -0.0348], [0.4611, -0.8264, 0.3052, 0.1062]]]

In [34]:
grain.betaDeviations

[1.8044684029575793, 0.9853812036538011, 1.283613169503778]

In [35]:
grain.variantCount

[0, 3, 0, 0, 0, 1]

In [36]:
possibleBetaOris = [item for sublist in grain.possibleBetaOris for item in sublist]

directions = [
    np.array([1,0,0]), 
    np.array([0,1,0]), 
    np.array([0,0,1])
]
markerSize = 100

fig, axes = plt.subplots(1, len(directions))

for direction, ax in zip(directions, axes):
    plot = Quat.plotIPF(grain.betaOris, direction, "cubic", marker='o', s=markerSize, fig=fig, ax=ax)
    Quat.plotIPF(possibleBetaOris, direction, "cubic", s=markerSize, plot=plot)

fig.tight_layout()

### Find the most common variant for each grain and set this as the beta orientaion

In [37]:
modeVariants = []
parentBetaOris = []

for grain in EbsdMap:
#     modeVariant = np.argmax(grain.variantCount)
#     parentBetaOri = grain.betaOris[modeVariant]
    
    variantCount = np.array(grain.variantCount)
    modeVariant = np.where(variantCount == np.max(variantCount))[0]
    if len(modeVariant) == 1:
        modeVariant = modeVariant[0]
        parentBetaOri = grain.betaOris[modeVariant]
    else:
        # multiple variants with same max
        modeVariant = -1
        parentBetaOri = Quat(1., 0., 0., 0.)
    
    modeVariants.append(modeVariant)
    grain.modeVariant = modeVariant
    parentBetaOris.append(parentBetaOri)
    grain.parentBetaOri = parentBetaOri

### Plot

In [32]:
plot = EbsdMap.plotGrainDataMap(grainData=modeVariants, vmin=-1, vmax=5, cmap="Set1")
plot.addColourBar("mode variant")

In [39]:
directions = [
    np.array([1,0,0]),
    np.array([0,1,0]),
    np.array([0,0,1])
]

fig, axes = plt.subplots(1, len(directions), figsize=(15,5))

for ax, direction in zip(axes, directions):
    betaGrainIPFColours = Quat.calcIPFcolours(parentBetaOris, direction, "cubic")

    EbsdMap.plotGrainDataMap(grainData=betaGrainIPFColours, fig=fig, ax=ax)
    
fig.tight_layout()